In [1]:
import torch
import random
import numpy as np
import datetime as dt
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torch.optim.lr_scheduler import MultiStepLR
from utils.util import icarl_cifar100_augment_data, get_dataset_per_pixel_mean

from avalanche.models import IcarlNet, make_icarl_net, initialize_icarl_net
from avalanche.training.supervised import ICaRL
from avalanche.logging import InteractiveLogger, TensorboardLogger
from avalanche.benchmarks.datasets import CIFAR100
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks.utils import AvalancheDataset, make_classification_dataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceTime, EpochAccuracy, StreamAccuracy

OSError: /usr/lib/aarch64-linux-gnu/libgomp.so.1: cannot allocate memory in static TLS block

In [ ]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
cudnn.enabled = False # cudnn library를 사용하지 않게 만듬 / cudnn (cuda 라이브러리)의 기본 설정이 시드를 고정하지 않기 때문에 고정된 결과를 얻을 수 있습니다. 그런데 cudnn을 사용하지 않으면 속도가 많이 느려집니다.
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용

In [ ]:
train_set = CIFAR100('data/CIFAR100', train=True, download=True)
test_set = CIFAR100('data/CIFAR100', train=False, download=True)

per_pixel_mean = get_dataset_per_pixel_mean(CIFAR100('data/CIFAR100', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()])))

transforms_group = dict(
        eval=(
            transforms.Compose(
                [
                    transforms.ToTensor(),
                    lambda img_pattern: img_pattern - per_pixel_mean,
                ]
            ),
            None,
        ),
        train=(
            transforms.Compose(
                [
                    transforms.ToTensor(),
                    lambda img_pattern: img_pattern - per_pixel_mean,
                    icarl_cifar100_augment_data,
                ]
            ),
            None,
        ),
    )

train_set = CIFAR100('data/CIFAR100', train=True,download=True,)
test_set = CIFAR100('data/CIFAR100', train=False,download=True,)
train_set = make_classification_dataset(train_set, transform_groups=transforms_group, initial_transform_group="train",)
test_set = make_classification_dataset(test_set, transform_groups=transforms_group, initial_transform_group="eval",)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_class = 100
incremental = 10
lr_milestones = [20,30,40,50]
lr_factor = 5.0
fixed_class_order = [87, 0, 52, 58, 44, 91, 68, 97, 51, 15, 94, 92, 10, 72, 49, 78, 61, 14, 8, 86, 84, 96, 18, 24, 32, 45, 88, 11, 4, 67, 69, 66, 77, 47, 79, 93, 29, 50, 57, 83, 17, 81, 41, 12, 37, 59, 25, 20, 80, 73, 1, 28, 6, 46, 62, 82, 53, 9, 31, 75, 38, 63, 33, 74, 27, 22, 36, 3, 16, 21, 60, 19, 70, 90, 89, 43, 5, 42, 65, 76, 40, 30, 23, 85, 2, 95, 56, 48, 71, 64, 98, 13, 99, 7, 34, 55, 54, 26, 35, 39]

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=False,
                        seed=seed,
                        shuffle=False,
                        fixed_class_order=fixed_class_order
                        )

model: IcarlNet = make_icarl_net(num_classes=num_class)
model.apply(initialize_icarl_net)

optimizer = optim.SGD(model.parameters(), lr=2.0, momentum=0.9, weight_decay=1e-5)
sched = LRSchedulerPlugin(MultiStepLR(optimizer, lr_milestones, gamma=1.0 / lr_factor))

In [ ]:
date = dt.datetime.now()
date = date.strftime("%Y_%m_%d_%H_%M_%S")

interactive_logger = InteractiveLogger()
tensor_logger = TensorboardLogger("logs_Jetson_iCaRL_cifar100_" + date)
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    StreamAccuracy(),
    loggers=[interactive_logger, tensor_logger])

In [ ]:
memory_size = 2000
train_batch = 512
eval_batch = 256
epoch = 60

buffer_transform = transforms.Compose([icarl_cifar100_augment_data])

strategies = ICaRL(model.feature_extractor, model.classifier, optimizer, memory_size, buffer_transform=buffer_transform, fixed_memory=True, train_mb_size=train_batch, train_epochs=epoch, eval_mb_size=eval_batch, device=device, plugins=[sched], evaluator=eval_plugin)  # criterion = ICaRLLossPlugin()

In [ ]:
for i, exp in enumerate(scenario.train_stream):
    eval_exps = [e for e in scenario.test_stream][: i + 1]
    strategies.train(exp)
    strategies.eval(eval_exps)